In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [68]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 60

In [69]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [70]:
def select_season (df, season):
    return df[df['season']==season]

In [71]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [72]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [73]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [74]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]

   
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]

    return X_train, Y_train, X_test, Y_test

In [75]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        #print(method.score(X_test, Y_test))
        #print('hi')
        lis.append(float(method.score(X_test, Y_test)))

    tmp[method_name]=lis 
    
    return tmp

In [76]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [77]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
    
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [78]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

# Loading data

In [79]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [80]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [81]:
n=4
m=4

In [82]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [83]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


# Parameter

In [380]:
season_to_be_predicted='2015/2016'
column_to_use_as_prediction=[47,49,53,39]
column_to_predict='Goal/No_Goal'

## SVM_LINEAR

In [381]:
svm_linear = svm.SVC(kernel='linear')
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [382]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [383]:
#c=Predict_season_with_current_season(allchampionships[0].df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear')

In [384]:
a['svm_linear'].mean()

0.601955782312925

In [385]:
b['svm_linear'].mean()

0.601955782312925

## SVM_RBF


In [398]:
svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf')

In [399]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf')

In [400]:
a['svm_rbf'].mean()

0.6090986394557824

In [401]:
b['svm_rbf'].mean()

0.5960034013605443

## CART

In [402]:
dtc = DecisionTreeClassifier()
d=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

In [403]:
e=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

In [404]:
d['dtc']

0.5436224489795919

In [414]:
e

,dtc
0,0.600000
1,0.600000
2,0.600000
3,0.600000
4,0.400000
5,0.600000
6,0.600000
7,0.400000
8,0.600000
9,0.200000


In [416]:
a.join(e, how='outer')

,svm_rbf,dtc
0,0.600000,0.600000
1,0.600000,0.600000
2,0.400000,0.600000
3,0.600000,0.600000
4,0.400000,0.400000
5,0.400000,0.600000
6,0.800000,0.600000
7,0.400000,0.400000
8,0.800000,0.600000
9,0.600000,0.200000


# Adaboost Classifier 

In [394]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

In [395]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

In [396]:
a['bagb'].mean()

0.5382653061224489

In [397]:
b['bagb'].mean()

0.5005102040816326

# Gradient Boosting Classifier


In [372]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

In [373]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

In [374]:
a['gradb'].mean()

0.49651360544217693

In [375]:
b['gradb'].mean()

0.5230442176870749

# Random Forest

In [376]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf')

In [377]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf')

In [378]:
a['rdf'].mean()

0.5130102040816326

In [379]:
b['rdf'].mean()

0.5293367346938777

# Working on sureness

In [271]:
df=allchampionships[0].df
df=df[df['season']==season_to_be_predicted]


In [353]:
for i in range(7,30):
    X_train=df[df.H_ROUND<i].iloc[:,column_to_use_as_prediction]
    Y_train=df[df.H_ROUND<i][column_to_predict]
    X_test=df[df.H_ROUND==i].iloc[:,column_to_use_as_prediction]
    Y_test=df[df.H_ROUND==i][column_to_predict]
    svm_linear = svm.SVC(kernel='linear', decision_function_shape='ovr')
    svm_linear.fit(X_train,Y_train)
    print(svm_linear.decision_function(X_test))
    #print(svm_linear.predict(X_test))
    #print(svm_linear.score(X_test,Y_test))
    #print(Y_test)
    

[ 4.00516375  1.5356142   5.08655979  7.64439174  2.86741613]
[-1.86862377  0.1554683  -3.30403984 -0.20296287  2.35014921]
[-2.09752508 -1.29698329  0.61701473  0.64274982 -0.27649069]
[-0.91138175  0.47347893 -0.41670361 -0.74808872  1.75798105]
[-1.44589231  0.32509975  1.3390916  -0.80986692 -0.68169027]
[ 0.68324007  0.33924769 -0.58582293 -0.71246161 -0.13906449]
[ 0.22398063 -0.21292635  0.85594089 -0.22130822 -0.49276205]
[ 0.68723481 -0.93427316  0.36507649  0.2487265  -0.48929326]
[ 0.57517344  0.57369524  0.44598612  1.90405536  1.6911471 ]
[ 1.00653217  0.89974712  0.96481827  0.99967182  1.03431154]
[ 0.5477124   0.48371137 -0.03535374  0.64082663  0.20393193]
[ 1.10896132  0.39520563  0.12864428  0.72163976  0.23799425]
[ 0.46493359  0.99922331  0.48515606  1.24607179  0.63493182]
[ 1.1039972   0.94859264  0.47072921  0.11382096]
[ 0.06357108  0.45532063  1.30508876 -0.64659682  0.87978582  0.90706822
  1.04808592]
[ 0.71267216  0.74234956  0.7537781   1.87275897  1.05076

In [349]:
svm_linear.decision_function(X_test)

array([ 0.70918747,  0.66477063,  0.71625791,  0.75144355,  0.95903544])

In [350]:
svm_linear.predict(X_test)

array([1, 1, 1, 1, 1])

In [351]:
svm_linear.score(X_test,Y_test)

0.80000000000000004

In [316]:
Y_test

1387    1
1388    0
1389    0
1390    0
1391    0
Name: Goal/No_Goal, dtype: int64

In [345]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
rdf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [346]:
rdf.feature_importances_

array([ 0.08142444,  0.09222371,  0.19321762,  0.17049799,  0.17541312,
        0.14064783,  0.14657529])

In [347]:
rdf.predict(X_test)

array([0, 1, 0, 1, 1])